# describe_duration.ipynb

This notebook provides descriptive statistics (median and IQR) of the duration of sick notes associated with long COVID.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns

from functools import reduce
from glob import glob

pd.options.mode.chained_assignment = None

## Import and Clean Data

In [2]:
# Function to parse string
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [3]:
# Read in and append input files
li = []

for file in glob('../output/cohorts/cohort_rates*.dta'):
    df_temp = pd.read_stata(file)
    # Creates date variable based on file name
    df_temp['cohort'] = file[find_nth(file, '_', 2)+1:-4]
    # Create population count
    df_temp['population'] = 1
    li.append(df_temp)
    
df_input = pd.concat(li, axis=0, ignore_index=False).reset_index(drop=True)

In [4]:
# Do not count those who had sick notes beyond end dates
df_input.loc[df_input.sick_note == 0, 'first_sick_note_duration'] = np.nan

In [5]:
# Columns to subset
subset_cols = ['cohort','age_group','sex',
               'ethnicity','imd','region_string',
               'first_sick_note_duration',
               'sick_note','population']

# Subset to relevant columns and do not count those beyond 
df_clean = df_input[subset_cols].rename(columns={'region_string':'region'})

_____

## Median & Mean

In [6]:
def compute_med_mean(path, demo=''):
    groups = ['cohort']
    if demo != '': 
        groups = ['cohort', demo]
    df_pct_ct = df_clean.groupby(
        groups)[['sick_note','population']].sum().reset_index()
    df_med = df_clean.groupby(
        groups)[['first_sick_note_duration']].apply(np.nanmedian).reset_index().rename(columns={0:'median_duration'})
    df_mean = df_clean.groupby(
        groups)[['first_sick_note_duration']].apply(np.nanmean).reset_index().rename(columns={0:'mean_duration'})
    df_pct25 = df_clean.groupby(
        groups)[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,25)).reset_index().rename(columns={0:'pct_25'})
    df_pct75 = df_clean.groupby(
        groups)[['first_sick_note_duration']].apply(lambda x: np.nanpercentile(x,75)).reset_index().rename(columns={0:'pct_75'})
    dfs = [df_pct_ct, df_med, df_mean, df_pct25, df_pct75]
    df_out = reduce(
        lambda left,right: pd.merge(left,right,on=groups), dfs
    )
    Path("../output/tabfig/").mkdir(parents=True, exist_ok=True)
    df_out.to_csv('../output/tabfig/' + path + '.csv', index=False)
    return df_out

In [7]:
# Overall
compute_med_mean('med_mean_overall')

,cohort,sick_note,population,median_duration,mean_duration,pct_25,pct_75
0,covid_2020,831.0,1381,272.0,281.884337,223.00,324.0
1,general_2019,4552.0,10000,272.0,276.043077,224.00,322.0
2,general_2020,6315.0,10000,271.0,275.224072,224.25,319.0
3,pneumonia_2019,193.0,457,274.5,277.255208,223.75,318.0


In [8]:
# Age group
compute_med_mean('med_mean_age_group', 'age_group')

,cohort,age_group,sick_note,population,median_duration,mean_duration,pct_25,pct_75
0,covid_2020,0-17,98.0,161,259.5,267.387755,211.25,314.50
1,covid_2020,18-24,87.0,147,271.5,280.755814,217.75,329.75
2,covid_2020,25-34,100.0,170,265.5,280.670000,230.50,316.50
3,covid_2020,35-44,82.0,146,274.0,271.634146,231.75,308.25
4,covid_2020,45-54,148.0,242,271.5,293.047297,221.00,339.25
5,covid_2020,55-69,151.0,253,277.0,278.576159,231.50,316.50
6,covid_2020,70-79,82.0,138,284.0,283.890244,218.25,331.75
7,covid_2020,80+,83.0,124,274.0,295.891566,227.00,342.00
8,general_2019,0-17,433.0,967,261.0,265.161663,217.00,319.00
9,general_2019,18-24,451.0,978,265.0,266.000000,218.25,307.00


In [9]:
# Sex
compute_med_mean('med_mean_sex', 'sex')

,cohort,sex,sick_note,population,median_duration,mean_duration,pct_25,pct_75
0,covid_2020,F,419.0,687,270.0,282.885167,223.00,325.00
1,covid_2020,M,412.0,694,276.0,280.868932,224.00,323.25
2,general_2019,F,2365.0,5143,272.0,277.201439,223.00,322.00
3,general_2019,M,2187.0,4857,273.0,274.791495,224.50,322.00
4,general_2020,F,3155.0,5008,271.0,276.410476,227.00,320.00
5,general_2020,M,3160.0,4992,271.0,274.039924,222.00,318.25
6,pneumonia_2019,F,92.0,212,273.0,275.163043,223.75,311.25
7,pneumonia_2019,M,101.0,245,275.5,279.180000,223.00,320.25


In [10]:
# Ethnicity
compute_med_mean('med_mean_ethnicity', 'ethnicity')

,cohort,ethnicity,sick_note,population,median_duration,mean_duration,pct_25,pct_75
0,covid_2020,White,475.0,821,273.0,286.305907,225.00,330.75
1,covid_2020,Asian or Asian British,78.0,101,264.0,273.294872,218.00,334.00
2,covid_2020,Other,64.0,110,256.0,257.234375,213.25,291.00
3,covid_2020,Unknown,214.0,349,278.5,282.593458,230.25,319.75
4,general_2019,White,2670.0,5956,273.0,277.850131,224.00,323.00
5,general_2019,Asian or Asian British,349.0,772,276.0,273.994269,228.00,324.00
6,general_2019,Other,361.0,772,276.0,269.235457,232.00,325.00
7,general_2019,Unknown,1172.0,2500,268.0,274.633646,221.00,316.00
8,general_2020,White,3769.0,5955,270.0,274.888889,224.00,317.00
9,general_2020,Asian or Asian British,495.0,785,269.5,274.813765,223.00,323.00


In [11]:
# IMD
compute_med_mean('med_mean_imd', 'imd')

,cohort,imd,sick_note,population,median_duration,mean_duration,pct_25,pct_75
0,covid_2020,0,52.0,77,277.5,275.403846,227.50,306.75
1,covid_2020,1,158.0,281,272.5,280.088608,219.25,318.50
2,covid_2020,2,153.0,242,271.0,280.052288,224.00,323.00
3,covid_2020,3,144.0,240,278.5,293.076389,232.25,345.25
4,covid_2020,4,158.0,269,259.5,271.588608,206.25,317.75
5,covid_2020,5,166.0,272,272.0,287.436364,229.00,331.00
6,general_2019,0,239.0,520,267.0,277.041841,229.00,319.00
7,general_2019,1,819.0,1866,270.0,272.373317,217.00,316.00
8,general_2019,2,850.0,1892,269.0,272.556471,222.00,317.00
9,general_2019,3,904.0,1873,271.5,276.719027,226.00,324.00


In [12]:
# Region
compute_med_mean('med_mean_region', 'region')

,cohort,region,sick_note,population,median_duration,mean_duration,pct_25,pct_75
0,covid_2020,East,77.0,142,266.0,283.675325,221.00,314.00
1,covid_2020,East Midlands,83.0,128,270.0,282.734940,225.50,319.00
2,covid_2020,London,154.0,257,269.5,278.428571,214.25,334.25
3,covid_2020,North East,78.0,121,286.0,285.858974,246.25,318.25
4,covid_2020,North West,91.0,137,286.0,293.755556,235.25,329.25
5,covid_2020,South East,80.0,147,259.5,281.400000,214.50,315.00
6,covid_2020,South West,93.0,165,281.0,290.086022,230.00,339.00
7,covid_2020,West Midlands,78.0,138,277.5,293.564103,230.50,328.75
8,covid_2020,Yorkshire and The Humber,97.0,146,263.0,254.154639,209.00,309.00
9,general_2019,East,452.0,1005,274.5,273.856195,221.50,324.00
